0 导入包

In [2]:
import pandas as pd
import json
from datasets import load_dataset

1 加载数据

In [29]:
# 查看数据集
df = pd.read_csv('0ori_data/csv_data.csv')
df

,订单号,商品id,商品编码,下单时间,商品名称,数量,支付价格,成本价,采购成本价
0,2023083193163,6971973442200,6971973442200,2023-08-31 23:00:55,禧遇棉花糖,1,2.5,1.54,NaN
1,2023083141300,221055484,221055484,2023-08-31 22:59:39,好丽友薯愿小包装,1,4.3,2.25,NaN
2,2023083141300,6972946670095,6972946670095,2023-08-31 22:59:39,徐多谷北海道豆制品40g,1,1.2,0.70,NaN
3,2023083141300,6958621301096,6958621301096,2023-08-31 22:59:39,真之味鸡蛋干,2,2.4,1.42,NaN
4,2023083141300,6948004703494,6948004703494,2023-08-31 22:59:39,鲜引力甘栗仁小包装,1,5.5,3.70,NaN
...,...,...,...,...,...,...,...,...,...
9995,2023082772991,6927462202091,6927462202091,2023-08-27 11:57:25,金锣王特级火腿肠80g,1,2.5,1.60,NaN
9996,2023082780198,221055372,221055372,2023-08-27 11:57:11,卡尔顿鸡蛋松饼,2,5.9,3.74,NaN
9997,2023082753252,6920698400378,6920698400378,2023-08-27 11:56:47,康师傅红烧牛肉面（BIG桶）,1,6.0,4.64,NaN
9998,2023082753252,6920698433543,6920698433543,2023-08-27 11:56:47,康师傅酸辣牛肉面（BIG桶）,1,6.0,4.64,NaN


2 数据筛选

In [30]:
df = df.groupby('订单号').agg(list) # 按照订单号合并，添加列表。因为有些订单包含多种商品
df.reset_index(inplace=True) # 重置索引
df.drop(['订单号', '商品id','商品编码','下单时间','成本价','采购成本价'], axis=1,inplace=True) # 删除不需要的列
df

,商品名称,数量,支付价格
0,[农夫山泉东方树叶],[3],[11.7]
1,"[脉动维生素饮料, 奥尔良鸡腿堡 140g]","[1, 1]","[4.9, 6.5]"
2,"[a1水果吐司, 桃李鸡蛋香松面包, a1水果吐司]","[1, 1, 1]","[2.5, 4.5, 2.5]"
3,[香飘飘果汁茶],[2],[11.8]
4,"[桃李醇熟切片面包, 伊利牛奶片]","[1, 1]","[8.5, 1.9]"
...,...,...,...
4255,"[雀巢咖啡丝滑即饮装系列268ml, 可乐500ml（无糖款）, 元气森林纤茶]","[1, 1, 1]","[5.8, 2.2, 5.8]"
4256,[桃李酵母系列],[1],[4.0]
4257,"[泰国小老板香脆海苔卷, 兰芳园冻柠茶500ml, 农夫山泉东方树叶]","[1, 1, 1]","[13.9, 7.3, 7.5]"
4258,"[统一酱拌面, 金锣清真鸡肉肠120g]","[1, 1]","[7.5, 2.5]"


3 构造qa数据集

In [32]:
# 数据处理逻辑
print(df.iloc[2]['商品名称'])
print(df.iloc[2]['数量'])
print(df.iloc[2]['支付价格'])

dict = {
    "input":f"价格区间：{int(sum(df.iloc[2]['支付价格']))}元，商品名称：{'，'.join(df.iloc[2]['商品名称'])}",
    "output":f"总价：{sum(df.iloc[2]['支付价格'])}元，{'，'.join(['{}个{}'.format(val1, val2) for val1, val2 in zip(df.iloc[2]['数量'], df.iloc[2]['商品名称'])])}"
}
dict

['a1水果吐司', '桃李鸡蛋香松面包', 'a1水果吐司']
[1, 1, 1]
[2.5, 4.5, 2.5]


{'input': '价格区间：9元，商品名称：a1水果吐司，桃李鸡蛋香松面包，a1水果吐司',
 'output': '总价：9.5元，1个a1水果吐司，1个桃李鸡蛋香松面包，1个a1水果吐司'}

In [33]:
# 批量处理数据
all_dict = []
for i in range(len(df)):
    dict = {
        "input":f"价格区间：{int(sum(df.iloc[i]['支付价格']))}元，商品名称：{'，'.join(df.iloc[i]['商品名称'])}",
        "output":f"总价：{sum(df.iloc[i]['支付价格'])}元，{'，'.join(['{}个{}'.format(val1, val2) for val1, val2 in zip(df.iloc[i]['数量'], df.iloc[i]['商品名称'])])}"
    }
    all_dict.append(dict)

In [34]:
all_dict

[{'input': '价格区间：11元，商品名称：农夫山泉东方树叶', 'output': '总价：11.7元，3个农夫山泉东方树叶'},
 {'input': '价格区间：11元，商品名称：脉动维生素饮料，奥尔良鸡腿堡 140g',
  'output': '总价：11.4元，1个脉动维生素饮料，1个奥尔良鸡腿堡 140g'},
 {'input': '价格区间：9元，商品名称：a1水果吐司，桃李鸡蛋香松面包，a1水果吐司',
  'output': '总价：9.5元，1个a1水果吐司，1个桃李鸡蛋香松面包，1个a1水果吐司'},
 {'input': '价格区间：11元，商品名称：香飘飘果汁茶', 'output': '总价：11.8元，2个香飘飘果汁茶'},
 {'input': '价格区间：10元，商品名称：桃李醇熟切片面包，伊利牛奶片',
  'output': '总价：10.4元，1个桃李醇熟切片面包，1个伊利牛奶片'},
 {'input': '价格区间：14元，商品名称：麻辣王子地道辣条，麻辣王子地道辣条，统一小当家干脆面系列，卫龙大面筋65g',
  'output': '总价：14.0元，6个麻辣王子地道辣条，3个麻辣王子地道辣条，1个统一小当家干脆面系列，1个卫龙大面筋65g'},
 {'input': '价格区间：2元，商品名称：印迹脆骨烤肠 70g', 'output': '总价：2.0元，1个印迹脆骨烤肠 70g'},
 {'input': '价格区间：0元，商品名称：维达4层手帕纸', 'output': '总价：0.8元，1个维达4层手帕纸'},
 {'input': '价格区间：13元，商品名称：11.5度老雪花500ml大听，雪碧500ml，农夫山泉东方树叶',
  'output': '总价：13.5元，1个11.5度老雪花500ml大听，1个雪碧500ml，1个农夫山泉东方树叶'},
 {'input': '价格区间：19元，商品名称：怡宝纯净水 555ml，外星人电解质水500ml，雀巢咖啡丝滑即饮装系列268ml，洽洽香瓜子160g',
  'output': '总价：19.94元，1个怡宝纯净水 555ml，1个外星人电解质水500ml，1个雀巢咖啡丝滑即饮装系列268ml，1个洽洽香瓜子160g'},
 {'inpu

4 保存数据

In [35]:
# 保存数据
json_data = json.dumps(all_dict,ensure_ascii=False)
with open('1qa_data/qa_data.jsonl', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

In [36]:
# 加载数据
dataset = load_dataset("json", data_files="1qa_data/qa_data.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
# 查看数据
dataset[0]

{'input': '价格区间：11元，商品名称：农夫山泉东方树叶', 'output': '总价：11.7元，3个农夫山泉东方树叶'}

5 构造商品价格信息

In [38]:
# 另外要查看商品的真实价格，看看模型是否能学习到
price = {}
for i in range(len(df)):
    x = [(a,b,c) for a, b ,c in zip(df.iloc[i]['数量'], df.iloc[i]['支付价格'],df.iloc[i]['商品名称'])]
    price[x[0][2]] = x[0][1]/x[0][0]
price

{'农夫山泉东方树叶': 3.9,
 '脉动维生素饮料': 4.9,
 'a1水果吐司': 2.5,
 '香飘飘果汁茶': 5.9,
 '桃李醇熟切片面包': 8.5,
 '麻辣王子地道辣条': 1.2,
 '印迹脆骨烤肠 70g': 2.0,
 '维达4层手帕纸': 0.8,
 '11.5度老雪花500ml大听': 3.8,
 '怡宝纯净水 555ml': 1.5,
 '怡宝1.555L': 2.5,
 '无糖可口可乐888ml': 3.5,
 '农夫山泉矿泉水550ml': 1.2,
 '好帮佳啵皮软糖25g': 1.2,
 '君乐宝老酸奶139g': 3.9,
 '丸美龙凤纯肉肠80g': 4.0,
 'vita维他气泡柠檬茶 250ml': 3.42,
 '红牛维生素功能饮料': 4.9,
 '大口饭团系列160g': 7.5,
 '珍珍荔枝饮料听装330ml': 2.4,
 '回味一梦手撕风干鸭脖65g': 8.9,
 '康师傅开心桶红烧牛肉面(桶装)': 4.5,
 '百岁山矿泉水570ml': 1.99,
 '劲霸面筋 100g': 4.5,
 '禧遇橡皮糖': 2.7,
 '百岁山矿泉水348ml': 1.7,
 '中街豆地主': 1.5,
 '可乐500ml（无糖款）': 2.2,
 '印迹原味烤肠 78g': 2.0,
 '百事可乐原味500ml': 2.8,
 '徽之梦果仁系列': 5.5,
 '依能果味饮料1L': 3.99,
 '金鸽多味葵花籽260g': 13.5,
 '依能无糖苏打水饮料900ml': 3.99,
 '伊利畅轻系列250g': 7.5,
 '上海梅林午餐肉罐头340g': 15.9,
 '娃哈哈冰红茶': 2.8,
 'CHALI茶里饮料390ml': 5.5,
 '果子熟了多多茶系列': 5.4,
 '雪碧2L': 6.5,
 '农夫山泉饮用天然水1.5L': 2.37,
 '太平梳打饼干': 4.56,
 'NFC果汁300ml': 8.3,
 '雪碧500ml': 2.2,
 '德氏大冰荔': 1.5,
 '不二家可吸果冻156g': 8.5,
 '尖叫饮料': 4.9,
 '茶π500ml': 4.9,
 '特仑苏纯牛奶苗条装250ml': 5.5,
 '卫龙魔芋爽24g': 1.5,
 '可乐888ml': 3

In [39]:
# 保存数据
price_data = json.dumps(price,ensure_ascii=False)
with open('0ori_data/price.json', 'w', encoding='utf-8') as json_file:
    json_file.write(price_data)

In [40]:
# 加载
with open('0ori_data/price.json','r', encoding='utf-8') as f:
    price = json.load(f)
price['小李传说牛排40g']

2.8